### 1. Import Packages

In [1]:
# Librerie di base
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Librerie per la Data Imputation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer

# Librerie per la Hyperparameters Optimization
from sklearn.model_selection import GridSearchCV

# Librerie per il Machine Learning
from sklearn.cross_decomposition import PLSRegression
import sklearn.metrics as metrics
from sklearn.model_selection import KFold, train_test_split, cross_val_score

# Librerie per lo scaling dei dati
from sklearn.preprocessing import StandardScaler

# Librerie per la Features Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

print('All packages successfully loaded')

All packages successfully loaded


### 2. Load Data & Peak Sheet

In [2]:
df = pd.read_excel('../../../data/ST000369.xlsx')

# Rimuoviamo le colonne che non ci servono
df.drop(columns=["Idx", "SampleID", "Class"], inplace=True)

# Visualizziamo le prime 5 righe del dataset
df.head()

,SampleType,HealthState,SmokingStatus,Sex,M1,M2,M3,M4,M5,M6,...,M172,M173,M174,M175,M176,M177,M178,M179,M180,M181
0,Plasma,Adenocarcinoma,Former,F,194,168,77,105985,118,15489,...,195,1606,300,108,7203,43,59,1009,383,68
1,Plasma,Adenocarcinoma,Former,F,215,143,154,100462,133,13534,...,184,364,1364,160,11910,82,99,136,1021,165
2,Plasma,Adenocarcinoma,Current,F,104,67,45,75301,94,7390,...,189,157,884,73,6372,59,35,115,530,72
3,Plasma,Adenocarcinoma,Current,M,360,642,82,42097,84,50943,...,320,1621,461,104,14011,69,86,845,1309,127
4,Plasma,Adenocarcinoma,Current,M,96,137,95,112346,168,41987,...,84,769,266,158,18140,40,87,1213,1037,149


### 2.1 Data Cleaning

Per quanto riguarda il data cleaning del dataset, sono necessarie due tipologie di operazioni sui dati: 
1. Alcuni valori della feature "*HealthState*" sono scritti incorrettamente, perciò dobbiamo effettuarne la correzione.
2. Tre features binarie sono espresse attraverso stringhe e non valori binari (0 e 1), perciò dobbiamo fare la sostituzione.

In [3]:
# Effettuiamo la correzione dei valori errati
df["HealthState"] = df["HealthState"].str.replace('Adenocarcnoma', 'Adenocarcinoma')

# Eliminiamo le righe con valori nulli all'interno delle colonne "HealthState" e "Sex"
df = df.dropna(subset=["HealthState", "Sex"])

# Convertiamo i valori di natura categorica in valori numerici
df['Output'] = df['HealthState'].apply(lambda x: 1 if x in ['Adenosquamous', 'Adenocarcinoma'] else 0)
df['SmokingStatus'] = df['SmokingStatus'].apply(lambda x: 1 if x in ['Current'] else 0)
df['Sex'] = df['Sex'].apply(lambda x: 1 if x in ['F'] else 0)

# Visualizziamo le prime 5 righe del dataset
df.head()

,SampleType,HealthState,SmokingStatus,Sex,M1,M2,M3,M4,M5,M6,...,M173,M174,M175,M176,M177,M178,M179,M180,M181,Output
0,Plasma,Adenocarcinoma,0,1,194,168,77,105985,118,15489,...,1606,300,108,7203,43,59,1009,383,68,1
1,Plasma,Adenocarcinoma,0,1,215,143,154,100462,133,13534,...,364,1364,160,11910,82,99,136,1021,165,1
2,Plasma,Adenocarcinoma,1,1,104,67,45,75301,94,7390,...,157,884,73,6372,59,35,115,530,72,1
3,Plasma,Adenocarcinoma,1,0,360,642,82,42097,84,50943,...,1621,461,104,14011,69,86,845,1309,127,1
4,Plasma,Adenocarcinoma,1,0,96,137,95,112346,168,41987,...,769,266,158,18140,40,87,1213,1037,149,1


### 3. Extract X & Y

In [4]:
X = df.drop(columns=['SampleType', 'HealthState', 'Output'])
X_features_names = X.columns
y = df.Output

### 3.1 Data Imputation

In [11]:
# KNNImputer
imputer_knn = KNNImputer(n_neighbors=2)
imputer_knn.fit(X)
X[:] = imputer_knn.fit_transform(X)
X_knn = X.copy()

### 3.2 Train-test Split

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_knn)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### 4. Initial Model Build

In [13]:
# Definiamo il modello SVC con gli iperparametri di default
model = PLSRegression()

# Addestriamo il modello
model.fit(X_train, y_train)

# Eseguiamo le previsioni sui dati di test
y_pred = model.predict(X_test)

# Convertiamo le previsioni in etichette di classe utilizzando una soglia di 0.5
threshold = 0.5
y_pred_class = (y_pred[:, 0] > threshold).astype(int)

### 5. Initial Model Evalutation

In [14]:
# Valutiamo le prestazioni del modello con gli iperparametri di default
accuracy_knn = metrics.accuracy_score(y_test, y_pred_class)
precision_knn = metrics.precision_score(y_test, y_pred_class)
recall_knn = metrics.recall_score(y_test, y_pred_class)
f1_knn = metrics.f1_score(y_test, y_pred_class)
roc_auc_knn = metrics.roc_auc_score(y_test, y_pred_class)

print(f'Accuratezza: {accuracy_knn}')
print(f'Precision: {precision_knn}')
print(f'Recall: {recall_knn}')
print(f'F1-score: {f1_knn}')
print(f'ROC AUC: {roc_auc_knn}')

Accuratezza: 0.6060606060606061
Precision: 0.625
Recall: 0.5882352941176471
F1-score: 0.6060606060606061
ROC AUC: 0.6066176470588236


Le prestazioni iniziali sono estremamente deludenti: abbiamo un'accuratezza del 60%, una Precision del 62% e una Recall dell'58%. Proviamo con la K-Fold Cross Validation.

In [17]:
# Testiamo diverse configurazioni di K
max = 0
k_best = 0
for k in range(5, 11): 
    kfolds = KFold(n_splits=k, shuffle=True, random_state=42)
    model = PLSRegression()
    scores = cross_val_score(model, X_scaled, y, cv=kfolds)
    mean = np.mean(scores)
    if mean > max: 
        max = mean
        k_best = k
    print(f"K={k}, Accuratezza Media: {mean}")
    
print("-------------------------------------------")
print(f"K ottimale: {k_best}")
print(f"Accuratezza media massima: {max}")

K=5, Accuratezza Media: 0.2153379011263546
K=6, Accuratezza Media: 0.1601900159305797
K=7, Accuratezza Media: 0.15094222521898418
K=8, Accuratezza Media: 0.20462742669886302
K=9, Accuratezza Media: 0.18013245772527867
K=10, Accuratezza Media: 0.1913862211558515
-------------------------------------------
K ottimale: 5
Accuratezza media massima: 0.2153379011263546


### 6. Hyperparameters Optimization

L'**ottimizzazione degli iperparametri** è un passo fondamentale nello sviluppo di modelli predittivi robusti. Infatti, aderire ai parametri predefiniti impedisce ai modelli di raggiungere il massimo delle prestazioni. A tale scopo, utilizziamo la tecnica **Grid Search**.

### 6.1 K-Fold

Prima di effettuare l'ottimizzazione degli iperparametri, cerchiamo il numero di fold ideale per la K-Fold.

In [18]:
# Testiamo diverse configurazioni di K
max = 0
k_best = 0
for k in range(5, 11): 
    kfolds = KFold(n_splits=k, shuffle=True, random_state=42)
    model = PLSRegression()
    scores = cross_val_score(model, X_knn, y, cv=kfolds)
    mean = np.mean(scores)
    if mean > max: 
        max = mean
        k_best = k
    print(f"K={k}, Accuratezza Media: {mean}")

print(f"K ottimale: {k_best}")

K=5, Accuratezza Media: 0.21533790112635448
K=6, Accuratezza Media: 0.16019001593057938
K=7, Accuratezza Media: 0.1509422252189841
K=8, Accuratezza Media: 0.20462742669886302
K=9, Accuratezza Media: 0.1801324577252787
K=10, Accuratezza Media: 0.19138622115585152
K ottimale: 5


Il numero ottimale di fold è 7. 
Ora possiamo effettuare la K-fold Cross-Validation con il numero ottimale di fold.


In [11]:
# Creiamo l'oggetto KFold per la Cross-Validation con il numero di fold ottimale
kfolds = KFold(n_splits=k_best, shuffle=True, random_state=42)

# Creiamo il modello SVM con gli iperparametri ottimizzati
model = svm.SVC()

# Applichiamo la K-fold Cross-Validation
scores = cross_val_score(model, X_knn, y, cv=kfolds, scoring='accuracy')

# Visualizziamo i risultati della cross-validation
print(f'Accuratezza media: {scores.mean()}')
print(f'Deviazione standard: {scores.std()}')

Accuratezza media: 0.6019668737060042
Deviazione standard: 0.09357227548423523


### 6.2 KNN Imputator Optimization

Cerchiamo il numero ideale di neighbors per l'imputazione KNN.

In [12]:
knn_classifier = KNeighborsClassifier()

# Definiamo il dominio di ricerca
param_space = {'n_neighbors': (1, 20)}

# Creiamo l'oggetto BayesSearchCV
# 'n_iter' determina quanti set distinti di iperparametri verranno esplorati durante la ricerca
# 'cv' indica il numero di fold nell'esecuzione della cross-validation
grid_search = GridSearchCV(knn_classifier, param_space, cv=kfolds, scoring='accuracy', n_jobs=-1)

# Addestriamo il modello sui dati
grid_search.fit(X_knn, y)

# Visualizziamo i risultati dell'ottimizzazione bayesiana
print("Migliore configurazione di parametri:", grid_search.best_params_)

Migliore configurazione di parametri: {'n_neighbors': 1}


/Users/nova/Programming/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nova/Programming/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nova/Programming/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nova/Programming/anaconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
 

In [13]:
# KNNImputer
imputer_knn = KNNImputer(n_neighbors=grid_search.best_params_['n_neighbors'])
imputer_knn.fit(X)
X[:] = imputer_knn.fit_transform(X)
X_knn = X.copy()

### 6.3 Model Hyperparameters

In [14]:
# Qui di seguito effetuiamo il tuning degli iperparametri del modello

# Creiamo un nuovo modello XGBClassifier
model_2 = svm.SVC()

# Definiamo la griglia con i parametri da testare
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': [0.001, 0.01, 0.1],
    'class_weight': [None, 'balanced']
}

# Creiamo l'oggetto GridSearchCV
grid_search = GridSearchCV(model_2, param_grid, scoring='accuracy', cv=k_best)

# Eseguiamo la ricerca e cross-validation sui dati di addestramento
grid_search.fit(X_knn, y)

# Ottieniamo i migliori parametri
best_params = grid_search.best_params_

# Creiamo un nuovo modello con i migliori parametri
best_model = svm.SVC(**best_params)

# Addestriamo il nuovo modello sui dati di addestramento
best_model.fit(X_knn, y)

# Applichiamo la K-fold Cross-Validation
scores = cross_val_score(best_model, X_knn, y, cv=kfolds, scoring='accuracy')

In [15]:
print(f'Accuratezza media: {scores.mean()}')
print(best_params)

Accuratezza media: 0.8043478260869567
{'C': 0.1, 'class_weight': None, 'gamma': 0.001, 'kernel': 'linear'}


### 7. Feature Selection

La funzione **model.best_features_** non è disponibile per le SVM, di conseguenza utilizzeremo la classe **SelectKBest**. 

In [16]:
# Applichiamo SelectKBest per selezionare le k migliori features
k = 20
bestfeatures = SelectKBest(score_func=chi2, k=k)
X_knn_mtblt = X_knn.drop(columns=["SmokingStatus", "Sex"])
fit = bestfeatures.fit(X_knn_mtblt, y)

# Creiamo un dataframe per visualizzare i risultati
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_knn_mtblt.columns)

# Concateniamo i due dataframe per avere una visione più chiara
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']

# Visualizziamo le k migliori features
print(featureScores.nlargest(k,'Score'))

    Features          Score
75       M76  432573.751614
105     M106  254871.972587
44       M45  219264.822458
116     M117  188987.402013
160     M161  175257.474427
24       M25  129345.798569
114     M115   66708.578744
59       M60   62967.310792
32       M33   56798.618636
102     M103   44291.229431
8         M9   42229.355832
49       M50   29751.509099
99      M100   29557.670183
120     M121   28532.068515
40       M41   28140.530215
132     M133   27449.486512
38       M39   24300.573050
46       M47   23230.614215
66       M67   22125.647382
103     M104   21860.747846


Valutiamo il modello utilizzando le 30 features più importanti con **train_test_split**.

In [17]:
# Otteniamo i nomi delle k migliori features
top_features = featureScores.nlargest(k, 'Score')['Features'].tolist()

# Creiamo un nuovo DataFrame 
X_top = X_knn[top_features]

# Effettuiamo il train-test split
X_train_top, X_test_top, y_train, y_test = train_test_split(X_top, y, test_size=0.2, random_state=42)

# Addestriamo il modello   
best_model.fit(X_train_top, y_train)

# Eseguiamo le previsioni sui dati di test
predictions_top = best_model.predict(X_test_top)

# Valutiamo le prestazioni del modello
accuracy_top = metrics.accuracy_score(y_test, predictions_top)
precision_top = metrics.precision_score(y_test, predictions_top)
recall_top = metrics.recall_score(y_test, predictions_top)
f1_top = metrics.f1_score(y_test, predictions_top)
roc_auc_top = metrics.roc_auc_score(y_test, predictions_top)

# Stampiamo i risultati
print(f'Accuratezza: {accuracy_top}')
print(f'Precision: {precision_top}')
print(f'Recall: {recall_top}')
print(f'F1-score: {f1_top}')
print(f'ROC AUC: {roc_auc_top}')

print("------------------------------------------")
print(featureScores.nlargest(k,'Score')) 

Accuratezza: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 0.7058823529411765
F1-score: 0.6857142857142857
ROC AUC: 0.6654411764705883
------------------------------------------
    Features          Score
75       M76  432573.751614
105     M106  254871.972587
44       M45  219264.822458
116     M117  188987.402013
160     M161  175257.474427
24       M25  129345.798569
114     M115   66708.578744
59       M60   62967.310792
32       M33   56798.618636
102     M103   44291.229431
8         M9   42229.355832
49       M50   29751.509099
99      M100   29557.670183
120     M121   28532.068515
40       M41   28140.530215
132     M133   27449.486512
38       M39   24300.573050
46       M47   23230.614215
66       M67   22125.647382
103     M104   21860.747846


Con **train_test_split** le metriche sono molto basse. Proviamo con la K-Fold.

In [18]:
scores = cross_val_score(best_model, X_top, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

Dopo 35 minuti di runtime, abbiamo ottenuto un'accuratezza del 67%. Un k=30 potrebbe non essere l'ideale. Infatti, l'accuratezza con tutte le feature era di 80%. Valutando i k da 2 a 25, troviamo che il numero di feature ideale potrebbe essere 20, con un'accuratezza del 72%. Tuttavia, ciò non basta a eguagliare l'accuratezza del modello sul dataset completo.

In [ ]:
""" max_fs = 0 
k_fs = 0
best_features = []

for k in range(2, len(X_knn.columns)):
    top_features = SelectKBest(score_func=chi2, k=k).fit(X_knn, y).get_support(indices=True)
    X_top = X_knn.iloc[:, top_features]
    
    scores = cross_val_score(best_model, X_top, y, cv=kfolds, scoring='accuracy')
    mean = np.mean(scores)
    
    if mean > max_fs: 
        max_fs = mean
        k_fs = k
        best_features = X_knn.columns[top_features].tolist()

    print(f"K={k}, Accuratezza Media: {mean}")

print(f"K ottimale: {k_fs}")
print(f"Feature ottimali: {best_features}") """

K=2, Accuratezza Media: 0.5758281573498963
K=3, Accuratezza Media: 0.59472049689441
K=4, Accuratezza Media: 0.6262939958592133
K=5, Accuratezza Media: 0.6374223602484472
K=6, Accuratezza Media: 0.6076604554865425
K=7, Accuratezza Media: 0.6873706004140787
K=8, Accuratezza Media: 0.7062629399585921
K=9, Accuratezza Media: 0.6570910973084887
K=10, Accuratezza Media: 0.7122153209109731
K=11, Accuratezza Media: 0.712991718426501
K=12, Accuratezza Media: 0.6749482401656314
K=13, Accuratezza Media: 0.6689958592132506
K=14, Accuratezza Media: 0.6943581780538303
K=15, Accuratezza Media: 0.6814182194616977
K=16, Accuratezza Media: 0.7000517598343684
K=17, Accuratezza Media: 0.7067805383022775
K=18, Accuratezza Media: 0.7189440993788819
K=19, Accuratezza Media: 0.6444099378881988
K=20, Accuratezza Media: 0.7248964803312629
K=21, Accuratezza Media: 0.6876293995859213
K=22, Accuratezza Media: 0.6995341614906833


### 9. Results

I risultati della ricerca sono i seguenti:

In [ ]:
# Elenchiamo i risultati migliori

# Numero di fold
kfolds = KFold(n_splits=k_best, shuffle=True, random_state=42)

# Modello SVM 
model = svm.SVC(**best_params)

# Applichiamo la K-fold Cross-Validation
scores = cross_val_score(model, X_knn, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

# Se vogliamo applicare la Feature Selection
scores = cross_val_score(model, X_top, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

NameError: name 'KFold' is not defined